In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import warnings
import os

In [2]:
os.chdir('/Users/dhamada/Downloads/honey-bee-behavior/')
from bees_lifetimetracking_2018data import datafunctions

In [3]:
from bees_lifetimetracking_2018data import definitions_2018 as bd

In [4]:
import pandas as pd
import numpy as np
from matplotlib import colors
import seaborn as sns


year = 2018
startday = pd.Timestamp(year,7,16)  # actual, for 2018:   16 July  (cameras turned on)
endday = pd.Timestamp(year,10,9)    # actual, for 2018:   was 10 September, but is now 09 Oct
alldaytimestamps = pd.date_range(start=startday,end=endday,freq='D')
numdays = len(alldaytimestamps)  # just do to 43?
numbees = 4096  #  I think 4096 is the total number.  This is the total number of barcodes, NOT the total number of bees actually tracked
numsubstrates = 10 # 10 actual substrates, and then 0 for undefined
comb_daynums = np.array([ 0,  5, 10, 15, 20, 26, 30, 35, 40, 44, 65, 75, 85]) + 16 - startday.day  # days from start day, assuming start day is in July

# get bee cohort data, and just process these for a single one
cohort_data = pd.read_csv('/Users/dhamada/Downloads/honey-bee-behavior/bees_lifetimetracking_2018data/all_cohorts.csv')
cohort_colornames = np.unique(cohort_data['cohort'])
cohort_tagids = [np.array(cohort_data[cohort_data['cohort']==name]['beeID']) for name in cohort_colornames]
cohort_birthdates = np.array([pd.Timestamp(np.array(cohort_data['DOB'][cohort_data['cohort']==c])[0]) for c in cohort_colornames])


In [5]:
cohort_data

,Unnamed: 0,beeID,cohort,DOB,jdate_birth,round,colony
0,0,0,queen,1 july 2000,183,0,c186
1,0,1,pink,16 june 2018,18167,0,c1
2,1,2,pink,16 june 2018,18167,0,c1
3,2,3,pink,16 june 2018,18167,0,c1
4,3,4,pink,16 june 2018,18167,0,c1
...,...,...,...,...,...,...,...
5338,411,2267,pink2,29 august 2018,18241,1,c8
5339,412,2268,pink2,29 august 2018,18241,1,c8
5340,413,2269,pink2,29 august 2018,18241,1,c8
5341,414,2270,pink2,29 august 2018,18241,1,c8


In [94]:
#cohort_birthdates = np.array([pd.Timestamp(np.array(cohort_data['DOB'][cohort_data['cohort']==c])[0],freq='D') for c in cohort_colornames])

np.array([cohort_data['DOB'][cohort_data['cohort']==c] for c in cohort_colornames][0])

array(['2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 2018',
       '2 july 2018', '2 july 2018', '2 july 2018', '2 july 20

In [95]:
from bees_lifetimetracking_2018data import displayfunctions as bp
datafunctions.init(bd) 
bp.init(bd)

warnings.filterwarnings("ignore")

In [96]:
PATH = '/Users/dhamada/Downloads/honey-bee-behavior/'

In [97]:
def getExitFrameLocation(df):
    x_max = 6500
    x_min = 3500
    y_min = 3800
    y_max = 5400
    
    x = np.array(df['x']).astype(float)
    y = np.array(df['y']).astype(float)
    camera = np.array(df['camera']).astype(int)
    conv_factor = 1
    x_adjusted = x/conv_factor + (np.logical_not(camera).astype(int))*bd.xpixels
    y_adjusted = y/conv_factor

    
    x_bounds = (x_adjusted >= x_min) & (x_adjusted <= x_max)
    y_bounds = (y_adjusted >= y_min) & (y_adjusted <= y_max)
    
    df['in_exit_frame'] = (x_bounds & y_bounds).astype('int')
    
    return df

In [98]:
def crossedDanceFloor(df):
    x_max = 6000
    x_min = 4700
    y_min = 4350
    y_max = 5200
#     dance_floor={'top':4350,'bottom':5200,'right':6000,'left':4700}
    x = np.array(df['x']).astype(float)
    y = np.array(df['y']).astype(float)
    camera = np.array(df['camera']).astype(int)
    conv_factor = 1
    x_adjusted = x/conv_factor + (np.logical_not(camera).astype(int))*bd.xpixels
    y_adjusted = y/conv_factor

    
    x_bounds = (x_adjusted >= x_min) & (x_adjusted <= x_max)
    y_bounds = (y_adjusted >= y_min) & (y_adjusted <= y_max)
    
    df['cross_df'] = (x_bounds & y_bounds).astype('int')
    
    return df

In [99]:
def numOfDanceFloorVisits(df):
    dance = df.groupby(['uid','daynum'])['cross_df'].sum().to_frame(name='df_visits').reset_index()
    df = pd.merge(df,dance, on=['uid','daynum'], how='left')
    
    return df
    

In [100]:
def getExitDist(df):
    df['exit_dist'] = datafunctions.getexitdistance(df.x,df.y,df.camera)
    return df

In [101]:
def get5MinBounds(time):
    lower = int(time - 900) 
    upper = int(time + 900)
    return upper,lower

In [102]:
def getData(daynum,df):
    if daynum >= 50:
        datadir = PATH + 'bees_lifetimetracking_2018data/beetrajectories_days_050_to_085/'
    else:
        datadir = PATH + 'bees_lifetimetracking_2018data/beetrajectories_days_000_to_049/'

    filename = '{}beetrajectories_{:0>3}.hdf'.format(datadir,daynum)
    print(filename)
    temp = pd.read_hdf(filename)

    ### DID THE BEE LEAVE?
    dataDir = PATH + 'df_day1min_alldays/'
    dfday = pd.read_hdf('{}df_day1min_{:0>2}.hdf'.format(dataDir,daynum))
    numtimedivs = len(dfday['timedivision'].unique()) # minutes in a day
    day_uids, day_ages, all_inhive, bee_obs, bee_exitdist = datafunctions.get_inout_estimates(dfday,
                                                                                              numtimedivs=numtimedivs)

    leave_hive_times_list = [np.where(diffs==-1)[0] for diffs in np.diff(all_inhive) ]

    leave_hive_times_by_uid = []

    for uid,leave_hive_times in zip(day_uids,leave_hive_times_list):
        for leave_hive_time in leave_hive_times:
            leave_hive_times_by_uid.append([uid,leave_hive_time])
    df_leave_hive_events = pd.DataFrame(leave_hive_times_by_uid,columns=['uid','leave hive time'])

    frames_per_day = 24*60*60*3
    frames_per_timedivision = frames_per_day / numtimedivs
    df_leave_hive_events['leave hive framenum'] = df_leave_hive_events['leave hive time']*frames_per_timedivision

    leave_hive_times_by_uid = []
    for uid,leave_hive_times in zip(day_uids,leave_hive_times_list):
        for leave_hive_time in leave_hive_times:
            leave_hive_times_by_uid.append([uid,leave_hive_time])
    df_leave_hive_events = pd.DataFrame(leave_hive_times_by_uid,columns=['uid','leave hive time'])

    # translate time to framenum (assuming 3 frames per second)
    frames_per_day = 24*60*60*3
    frames_per_timedivision = frames_per_day / numtimedivs
    df_leave_hive_events['leave hive framenum'] = (df_leave_hive_events['leave hive time']*frames_per_timedivision).astype('int')


    temp['forage'] = np.where((temp['framenum'].isin(df_leave_hive_events['leave hive framenum']))&(temp['uid'].isin(df_leave_hive_events['uid'])), 1,0)
    temp['time'] = temp['framenum']/(3*60)
    
    temp["x_norm"] = 2 * math.pi * temp["time"] / temp["time"].max()
    temp["cos_x"] = np.cos(temp["x_norm"])
    temp["sin_x"] = np.sin(temp["x_norm"])
    
    ### AGE
    age_id = dfday[['Age','Bee unique ID']].astype('int')
    age_id = age_id.rename(columns={"Bee unique ID":'uid'})
    age_id = age_id.drop_duplicates()

    temp = pd.merge(temp,age_id,on='uid', how='left')

    ### Indicator of frame
    temp = getExitFrameLocation(temp)

    
    ### Exit Diastance
    temp = getExitDist(temp)
    
    ### Crossed Dance Floor
    temp = crossedDanceFloor(temp)
    
    ### Number of visits
    temp =  numOfDanceFloorVisits(temp)
    
    df = df.append(temp)
    
    return df
    

In [73]:
df2 = pd.DataFrame()
for x in range(20,26):
   df2 = getData(x,df2)

/Users/dhamada/Downloads/honey-bee-behavior/bees_lifetimetracking_2018data/beetrajectories_days_000_to_049/beetrajectories_020.hdf


FileNotFoundError: File /Users/dhamada/Downloads/honey-bee-behavior/df_day1min_alldays/df_day1min_20.hdf does not exist